# app.py

Responsible for the main routes of the website.

In [ ]:
from flask import Flask, render_template, request, redirect, url_for
from data import *

app = Flask(__name__)

In [ ]:
# defines a route for the main page ("/") that displays the "index.html" template when accessed.

@app.route('/')
def index():
    return render_template('index.html')

In [ ]:
# sets up a route for displaying movies based on a specific genre.
# When someone visits a URL like "/movies/action" (with "action" being the genre), the code retrieves 
# a list of movies belonging to that genre and displays them using a template called "movies.html".
# The genre and the list of movies are passed as variables to the template.

@app.route('/movies/<genre>')
def movies(genre):
    films_list = read_films_by_genre(genre)
    return render_template("movies.html", genre=genre, films=films_list)

In [ ]:
# a route to display a movie based on it "film_id", retrieving the movie information, if it uses such parameters,
# it is redirected to the "film.html"

@app.route('/movies/<int:film_id>')
def movie(film_id):
    film = read_films_by_film_id(film_id)
    if film:
        return render_template("film.html", film=film)

In [ ]:
# when routed in '/register', "register.html" is accessed.

@app.route('/register')
def register():
    return render_template('register.html')

In [ ]:
# sets up a route to process form data submitted through a POST request, extracts film information from the form, 
# inserts it into a film database, and redirects the user to the movies page filtered by the genre of the submitted film.

@app.route('/processed', methods=['post'])
def processing():
    film_data = {
        "genre": request.form['genre'],
        "title": request.form['film_title'],
        "year": request.form['film_year'],
        "cast": request.form['film_cast'],
        "director": request.form['film_director'],
        "summary": request.form['film_summary'],
        "url": request.form['film_url']
    }
    insert_film(film_data)
    return redirect(url_for('movies', genre=request.form['genre']))

In [ ]:
# used for editing or deleting the existing information in the website

@app.route('/modify', methods=['post'])
def modify():
    if request.form["modify"] == "edit":
        film_id = request.form["film_id"]
        film = read_films_by_film_id(film_id)
        return render_template('update.html', film=film)
    elif request.form["modify"] == "delete":
        film_id = request.form["film_id"]
        delete_film(film_id)
        return redirect(url_for('index'))

In [ ]:
# used to update film information
# retrieves the updated data and update the database

@app.route('/update', methods=['post'])
def update():
    film_data = {
        "genre": request.form['genre'],
        "title": request.form['film_title'],
        "year": request.form['film_year'],
        "cast": request.form['film_cast'],
        "director": request.form['film_director'],
        "summary": request.form['film_summary'],
        "url": request.form['film_url']
    }
    update_film(film_data)
    return redirect(url_for('film', film_id=request.form['film_id']))

In [ ]:
# this code is for retrieving information by searching a certain data, then it will be redirected to "search.html"

@app.route('/search', methods=['get'])
def search():
    query = request.args.get('query', '')
    results = search_film(query)
    return render_template('search.html', query=query, results=results)

In [ ]:
# used for debugging

if __name__ == "__main__":
    app.run(debug=True)

# data.py

responsible for defining the data that would be imported in the web application.

In [ ]:
# declares that data would come from sql database
import sqlite3

# declaring the specific db file that would be used
db_path = "md.db"

In [ ]:
# connects the code to the SQLite database that allow accessing rows as dictionaries

def connect_to_db(path):
    conn = sqlite3.connect(path)
    conn.row_factory = sqlite3.Row
    return (conn, conn.cursor())

In [ ]:
# defines the "read_films_by_genre" function and retrieves data that has matching genre

def read_films_by_genre(genre):
    conn, cur = connect_to_db(db_path)
    query = 'SELECT * FROM films WHERE genre = ?'
    value = genre
    results = cur.execute(query,(value,)).fetchall()
    conn.close()
    return results

In [ ]:
# defines a function that reads a film's information from a SQLite database based on its ID
# connects to the database, executes a query to retrieve the film with the specified ID

def read_films_by_film_id(film_id):
    conn, cur = connect_to_db(db_path)
    query = 'SELECT * FROM films WHERE id = ?'
    value = film_id
    result = cur.execute(query,(value,)).fetchone()
    conn.close()
    return result

In [ ]:
# defines the function that you can insert a new data to the database

def insert_film(film_data):
    conn, cur = connect_to_db(db_path)
    query = 'INSERT INTO films (genre, title, year, cast, director, summary, url) VALUES (?,?,?,?,?,?,?)'
    values = (film_data['genre'], film_data['title'], film_data['year'], 
              film_data['cast'], film_data['director'], film_data['summary'], 
              film_data['url'])
    cur.execute(query, values)
    conn.commit()
    conn.close()

In [ ]:
# defines the function that you can edit and change the existing data in the database and replace it with a new one

def update_film(film_data):
    conn, cur = connect_to_db(db_path)
    query = "UPDATE films SET genre=?, title=?, year=?, cast=?, director=?, summary=?, url=? WHERE id=?"
    values = (film_data['genre'], film_data['title'], film_data['year'],
              film_data['cast'], film_data['director'], film_data['summary'],
              film_data['url'], film_data['film_id'])
    cur.execute(query, values)
    conn.commit()
    conn.close()

In [ ]:
# defines the function to delete a data in the SQLite database

def delete_film(film_id):
    conn, cur = connect_to_db(db_path)
    query = "DELETE FROM films WHERE id = ?"
    values = (film_id,)
    cur.execute(query, values)
    conn.commit()
    conn.close()

In [ ]:
# this defines the function to search for what the user is/are looking for in the database

def search_film(query):
    conn, cur = connect_to_db(db_path)
    sql_query = "SELECT * FROM films WHERE genre LIKE ? OR title LIKE ?"
    value = "%{}%".format(query)
    results = cur.execute(sql_query, (value, value)).fetchall()
    conn.close()
    return results

# base.html

The backbone html file of the website. This is where all html files are connected

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>

# declaring the css format that will be used in the website
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0-alpha3/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-KK94CHFLLe+nY2dmCWGMq91rCGa5gtU4mk92HdvYe+M/SXH301p5ILy+dN9+nJOZ" crossorigin="anonymous">

# linking the css format files to the htmnl file via jinja
    <link rel="stylesheet" href="{{ url_for('static',filename ='css/style.css') }}">
    <link rel="stylesheet" href="{{ url_for('static',filename ='css/index.css') }}">
    <link rel="stylesheet" href="{{ url_for('static',filename ='css/update.css') }}">
    <link rel="stylesheet" href="{{ url_for('static',filename ='css/film.css') }}">
    <link rel="stylesheet" href="{{ url_for('static',filename ='css/movies.css') }}">
    <link rel="stylesheet" href="{{ url_for('static',filename ='css/register.css') }}">
    <link rel="stylesheet" href="{{ url_for('static',filename ='css/search.css') }}">
    
# text that would appear in the tab bar
    <title> Route 196 </title>
</head>

In [ ]:
# text and format that would appear in the navigation bar
    <nav class="navbar navbar-expand-lg navbar-light bg-warning">
        <div class="container-fluid">
          <a class="navbar-brand" href="{{url_for('index')}}"> Route 196 </a>

In [ ]:
# indicates that the navigation bar could be toggled and hidden in a button when it is in a smaller scale
          <button class="navbar-toggler" type="button" data-bs-toggle="collapse" data-bs-target="#navbarSupportedContent" aria-controls="navbarSupportedContent" aria-expanded="false" aria-label="Toggle navigation">
            <span class="navbar-toggler-icon"></span>
          </button>

In [ ]:
# this code indicates that the navigation bar would collapse its elements if it is in a smaller screen or size
# this code also indicate the urls connected and where it would be redirected if accessed.
<div class="collapse navbar-collapse" id="navbarSupportedContent">
            <ul class="navbar-nav me-auto mb-2 mb-lg-0">
              <li class="nav-item">
                <a class="nav-link active" aria-current="page" href="{{url_for('index')}}">Home</a>
              </li>
              
              <li class="nav-item">
                <a class="nav-link" href="{{url_for('register')}}">Share your Film</a>
              </li>
              
            </ul>
            <form class="d-flex" role="search">
              <input class="form-control me-2" type="search" placeholder="Search" aria-label="Search">
              <button class="btn btn-outline-success" type="submit">Search</button>
            </form>
          </div>
        </div>
      </nav>

    <div class="m-4">
        {% block content %}
        {% endblock %}    
    </div>

In [ ]:
# font that would be used in the website
	<script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0-alpha3/dist/js/bootstrap.bundle.min.js" integrity="sha384-ENjdO4Dr2bkBIFxQpeoTz1HIcje39Wm4jDKdf19U8gI4ddQ3GYNS7NTKfAdVQSZe" crossorigin="anonymous"></script>

</body>
</html>

# index.html

"index.html" file is responsible for the elements that will appear in the home page

In [ ]:
# this jinja indicates that the elements presented in this file will be connected in "base.html"
{% extends 'base.html' %}

# placeholder in the template that would overide the content in base.html
{% block content %}

In [ ]:
# header in the main page
<h1 class="rou_heading" align="center"> Route 196 </h1>

#text that would be seen under the header
<p align="center"> Browse through the site to look for your favorite films that you could watch on the go! </p><br>

In [ ]:
# buttons that would give you access to the different movie genre
<ul class="index">
    <button class='img-ac'><a href = "movies/action"> ACTION </a></button>
    <button class='img-co'><a href = "movies/comedy"> COMEDY </a></button>
    <button class='img-dr'><a href = "movies/drama"> DRAMA </a></button>
    <button class='img-hor'><a href = "movies/horror"> HORROR </a></button>
    <button class='img-rom'><a href="movies/romance"> ROMANCE </a></button>
    <button class='img-sci'><a href = "movies/sci-fi"> SCI-FI </a></button>
    <button class='img-sus'><a href = "movies/suspense"> SUSPENSE </a></button>
    <button class='img-thr'><a href = "movies/thriller"> THRILLER </a></button>
</ul>

# completion of the block and allows the template engine to replace or render the contents of the block
{% endblock %}

# film.html

contains the details that should be seen per film/movie

In [ ]:
# indicates that this html file is extended to "base.html"
{% extends 'base.html' %}

# placeholder in the template that would overide the content in base.html
{% block content %}

# shows the title of particular data in md.db
<h1 class="title" align="center">{{ film.title }}</h1>

# presents the image of a specific data in md.db
<img src="{{ film.url }}"><br>

# displays the attributes of the film
<ul class="film">
<li class="card" style="--color: black; --bg-color:#FFEDCB">
    <div class="info">Year Released</div>
    <div class="content">{{ film.year }}</div>
  </li>

  <li class="card" style="--color: black; --bg-color:#DDA01D">
    <div class="info">Cast</div>
    <div class="content">{{ film.cast }}</div>
  </li>

  <li class="card" style="--color: black; --bg-color:#D1EAE7">
    <div class="info">Director</div>
    <div class="content">{{ film.director }}</div>
  </li>

  <li class="card" style="--color: black; --bg-color:#F9F871">
    <div class="info">Summary</div>
    <div class="content">{{ film.summary }}</div>
  </li>
</ul>

# function used to redirect 
<form action="{{ url_for('modify') }}" method="post">
    <input type="hidden" name="film_id" value="{{ film.id }}">
    
    # button that will be used to edit the data in the website
    <input class="btn btn-secondary" type="submit" name="modify" value="edit">
    
    # button that will be used to delete the data in the website
    <input class="btn btn-danger" type="submit" name="modify" value="delete">
</form>

# completion of the block and allows the template engine to replace or render the contents of the block
{% endblock %}

# movies.html

this is an html file where the users will be redirected to once they selected a specific genre in the main/home page

In [ ]:
# indicates that this html file is extended to "base.html"
{% extends 'base.html' %}

# placeholder in the template that would overide the content in base.html
{% block content %}

# heading that is generated by the genre data in the database of md.db
<h1 class="m_h1" align="center">{{ genre }} Movies</h1><br>

<ul>
# presents the data in the datatable films
    {% for film in films %}
    <li>
    # generates button for every film that is added in the database
        <button class='m-button'>
            <a href="{{ url_for('movie', film_id=film.id) }}">{{ film.title }}</a>
        </button>
    </li>
    {% endfor %}
    # end of the condition for {% for film in films %}
</ul>

# completion of the block and allows the template engine to replace or render the contents of the block
{% endblock %}

# register.html

html file for adding new data in the website.

In [ ]:
# indicates that this html file is extended to "base.html"
{% extends 'base.html' %}

# placeholder in the template that would overide the content in base.html
{% block content %}

<h1 class="rh1"> Share Your Beloved Film! </h1>

# submits data to a particular url with the HTTP POST method
<form action="{{ url_for('processing') }}" method="post">

    <label class="reg" for="f_genre"> Genre: </label>
  # dropdown menu to choose the appropriate genre
    <select name="genre" id="f_genre" required>
        <option disabled selected value>...</option>
        <option value="action"> Action </option>
        <option value="comedy"> Comedy </option>
        <option value="drama"> Drama </option>
        <option value="horror"> Horror </option>
        <option value="romance"> Romance </option>
        <option value="sci-fi"> Sci-fi </option>
        <option value="suspense"> Suspense </option>
        <option value="thriller"> Thriller </option>
    </select><br><br>

    # attributes for the deatils that is needed for inserting a new data
    <label class="reg" for="f_title"> Movie Title: </label>
    <input type="text" name="film_title" id="f_title" required><br><br>
    
    <label class="reg" for="f_year"> Year Released: </label>
    <input type="number" name="film_year" id="f_year"><br><br>

    <label class="reg" for="f_cast"> Cast: </label>
    <input type="text" name="film_cast" id="f_cast"><br><br>

    <label class="reg" for="f_director"> Director: </label>
    <input type="text" name="film_director" id="f_director"><br><br>
    
    <label class="reg" for="f_url"> Movie Poster URL:</label>
    <input type="url" name="film_url" id="f_url"><br>
    
    <label class="reg" for="f_sum"> Summary: </label> <br>
    <textarea name="film_summary" placeholder="Tell something about your film..." cols="30" rows="10"></textarea><br>
    <input type="submit" value="Action!">
</form>

# completion of the block and allows the template engine to replace or render the contents of the block
{% endblock %}

# search.html

html file that is responsible for the search engine function

In [ ]:
# indicates that this html file is extended to "base.html"
{% extends 'base.html' %}

# placeholder in the template that would overide the content in base.html
{% block content %}
<div class="search">
  <h1>Search Results for "{{ query }}"</h1>
  {% if results %}
    <ul class="search-results">
    # presents the condition that should come up whenever the user uses the search bar
      {% for film in results %}
        <li>
          <h2>{{ film.genre }} - {{ film.title }}</h2>
          <img src="{{ film.url }}" alt="{{ film.genre }} {{ film.title }}">
          <p>Year Released: {{ film.year }}</p>
          <p>Cast: {{ film.cast }}</p>
          <p>Director: {{ film.director }}</p>
          <p>Summary: {{ film.summary }}</p>
        </li>
      {% endfor %}
    </ul>
  # another condition if the if statement was not met, it would present this message
  {% else %}
    <p>Oh no, the information that you're currently looking for is not available.</p>
  # end of if statement
  {% endif %}
</div>

# completion of the block and allows the template engine to replace or render the contents of the block
{% endblock %}

# update.html

html file responsible for update the existing data

In [ ]:
# indicates that this html file is extended to "base.html"
{% extends 'base.html' %}

# placeholder in the template that would overide the content in base.html
{% block content %}

<h1 class="reg-head"> Do you have some updates? </h1>

# submits data to the update url using the HTTP POST method
<form action="{{ url_for('update') }}" method="post">

    <input type="hidden" name="film_id" value="{{ film.id }}">
    <label class ="update" for="f_genre"> Genre: </label>
    <select name="genre" id="genre" required>
        <option disabled selected value>...</option>
        
# conditions for what option would be selected in the dropdown menu of the genre attribute
        {% if film.genre == "action" %}
        <option value="action" selected> Action </option>
        <option value="comedy"> Comedy </option>
        <option value="drama"> Drama </option>
        <option value="horror"> Horror </option>
        <option value="romance"> Romance </option>
        <option value="sci-fi"> Sci-fi </option>
        <option value="suspense"> Suspense </option>
        <option value="thriller"> Thriller </option>
        
        {% elif film.genre == "comedy" %}
        <option value="action"> Action </option>
        <option value="comedy" selected> Comedy </option>
        <option value="drama"> Drama </option>
        <option value="horror"> Horror </option>
        <option value="romance"> Romance </option>
        <option value="sci-fi"> Sci-fi </option>
        <option value="suspense"> Suspense </option>
        <option value="thriller"> Thriller </option>
        
        {% elif film.genre == "drama" %}
        <option value="action"> Action </option>
        <option value="comedy"> Comedy </option>
        <option value="drama" selected> Drama </option>
        <option value="horror"> Horror </option>
        <option value="romance"> Romance </option>
        <option value="sci-fi"> Sci-fi </option>
        <option value="suspense"> Suspense </option>
        <option value="thriller"> Thriller </option>

        {% elif film.genre == "horror" %}
        <option value="action"> Action </option>
        <option value="comedy"> Comedy </option>
        <option value="drama"> Drama </option>
        <option value="horror" selected> Horror </option>
        <option value="romance"> Romance </option>
        <option value="sci-fi"> Sci-fi </option>
        <option value="suspense"> Suspense </option>
        <option value="thriller"> Thriller </option>

        {% elif film.genre == "romance" %}
        <option value="action"> Action </option>
        <option value="comedy"> Comedy </option>
        <option value="drama"> Drama </option>
        <option value="horror"> Horror </option>
        <option value="romance" selected> Romance </option>
        <option value="sci-fi"> Sci-fi </option>
        <option value="suspense"> Suspense </option>
        <option value="thriller"> Thriller </option>

        {% elif film.genre == "sci-fi" %}
        <option value="action"> Action </option>
        <option value="comedy"> Comedy </option>
        <option value="drama"> Drama </option>
        <option value="horror"> Horror </option>
        <option value="romance"> Romance </option>
        <option value="sci-fi" selected> Sci-fi </option>
        <option value="suspense"> Suspense </option>
        <option value="thriller"> Thriller </option>

        {% elif film.genre == "suspense" %}
        <option value="action"> Action </option>
        <option value="comedy"> Comedy </option>
        <option value="drama"> Drama </option>
        <option value="horror"> Horror </option>
        <option value="romance"> Romance </option>
        <option value="sci-fi"> Sci-fi </option>
        <option value="suspense" selected> Suspense </option>
        <option value="thriller"> Thriller </option>

        {% elif film.genre == "thriller" %}
        <option value="action"> Action </option>
        <option value="comedy"> Comedy </option>
        <option value="drama"> Drama </option>
        <option value="horror"> Horror </option>
        <option value="romance"> Romance </option>
        <option value="sci-fi"> Sci-fi </option>
        <option value="suspense"> Suspense </option>
        <option value="thriller" selected> Thriller </option>

        # end of if and then statement
        {% endif %}
    </select>

    <br></br>

# different attributes that needs to be filled out for films
    <label class="update" for="f_title"> Movie Title: </label>
    <input type="text" name="film_title" id="f_title" value="{{ film.title }}" required><br><br>

    <label class="update" for="f_year"> Year: </label>
    <input type="number" name="film_year" id="f_year" value="{{ film.year }}"><br><br>

    <label class="update" for="f_cast"> Cast: </label>
    <input type="text" name="film_cast" id="f_cast" value="{{ film.cast }}"><br><br>

    <label class="update" for="f_director"> Director: </label>
    <input type="text" name="film_director" id="f_director" value="{{ film.director }}"><br><br>
    
    
    <label class="update" for="f_url"> Movie Poster URL: </label>
    <input type="url" name="film_url" id="f_url" value="{{ film.url }}"><br>
    
    <label class="update" for="f_sum"> Summary: </label><br>
    <textarea name="film_summary" cols="30" rows="10"> {{ film.summary }} </textarea><br>
    <input type="submit" value="Update">
</form>

# completion of the block and allows the template engine to replace or render the contents of the block
{% endblock %}